# Введение

<style>
a:link {
  color: green;
  background-color: transparent;
  text-decoration: none;
}
</style>

Данный ноутбук закрепляет на практике понимание архитектуры "трансформер", которая была предложена группой исследователей из Google в 2017 году в статье [Attention Is All You Need](https://arxiv.org/abs/1706.03762).

Трансформер и его модификации получили широкое распространение в задачах NLP ([BERT](https://arxiv.org/abs/1810.04805), [ALBERT](https://arxiv.org/abs/1909.11942v6), [RoBERTa](https://arxiv.org/abs/1907.11692), [GPT](https://cdn.openai.com/research-covers/language-unsupervised/language_understanding_paper.pdf), [GPT-2](https://d4mucfpksywv.cloudfront.net/better-language-models/language_models_are_unsupervised_multitask_learners.pdf), [GPT-3](https://arxiv.org/abs/2005.14165), [ruGPT-3](https://sbercloud.ru/ru/warp/gpt-3), [Turing-NLG](https://www.microsoft.com/en-us/research/blog/turing-nlg-a-17-billion-parameter-language-model-by-microsoft/), [Switch Transformer](https://arxiv.org/abs/2101.03961), [EFL](https://arxiv.org/abs/2104.14690v1), [Reformer](https://ai.googleblog.com/2020/01/reformer-efficient-transformer.html), [T5](https://arxiv.org/abs/1910.10683) и много других). Языковые модели, основанные на трансформерах, способны генерировать [тексты](https://mobile.twitter.com/raphamilliere/status/1289129723310886912), почти неотличимые от написанных человеком. Трансформеры также начинают активно использоваться в компьютерном зрении ([ViT](https://arxiv.org/abs/2106.04803v2), [HRNet](https://arxiv.org/abs/1909.11065v6), [SwinIR](https://arxiv.org/abs/2108.10257) и др., см. также [здесь](https://habr.com/ru/post/578308/) и [здесь](https://arxiv.org/abs/2101.01169)), в мультимодальных сетях, связанных одновременно с изображениями и текстом ([CLIP](https://openai.com/blog/clip/), [DALL-E](https://openai.com/blog/dall-e/), [Wu Dao](https://www.forbes.com/sites/alexzhavoronkov/2021/07/19/wu-dao-20bigger-stronger-faster-ai-from-china/) и др.), и даже в предсказании структуры белков ([AlphaFold2](https://www.nature.com/articles/s41586-021-03819-2)). Более того, есть [работа](https://arxiv.org/abs/2103.05247), показывающая, что трансформер может претендовать на роль универсальной архитектуры для одновременного решения самых разных задач.

Трансформер - непростая архитектура, и перед тем, как переходить к практике построения трансформеров, рекомендуется хорошо изучить принцип их устройства. Для этого вы можете использовать следующие материалы:

1. Материалы обучающего модуля курса NLP-инженер
1. [Детальное описание устройства трансформера](https://limitless-depths-73156.herokuapp.com/Attention_Is_All_You_Need) от автора данного ноутбука
1. Научную статью [Attention Is All You Need](https://arxiv.org/abs/1706.03762) (англ.)
1. Статью [The Illustrated Transformer](https://jalammar.github.io/illustrated-transformer/) (англ.) и [перевод на Хабре](https://habr.com/ru/post/486358/)
1. [Часть обучающего курса про трансформеры от Лены Войты](https://lena-voita.github.io/nlp_course/seq2seq_and_attention.html) (англ.)
1. [Практику по трансформерам от UvA Deep Learning Tutorials](https://uvadlc-notebooks.readthedocs.io/en/latest/tutorial_notebooks/tutorial6/Transformers_and_MHAttention.html) (англ.)
1. Экспертов поддержки в Slack, которым без вас скучно :)

# Часть 1

В этой части мы рассмотрим практическую реализацию механизма self-attention.

### Задание 1

Дан вариант реализации scaled dot-product self-attention с помощью матричных операций:

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

def scaled_dot_product_attention(Q, K, V, masked=False):
    '''Scaled dot product attention from https://arxiv.org/abs/1706.03762
    Args:
        Q - query, torch.Tensor of shape (batch_size, num_queries, key_size)
        K - keys, torch.Tensor of shape (batch_size, num_values, key_size)
        V - values, torch.Tensor of shape (batch_size, num_values, value_size)
        masked - [not implemented] boolean: should we use attention mask?
    Returns:
        torch.Tensor of shape (batch_size, num_queries, value_size)
    '''
    assert Q.ndim == K.ndim == V.ndim == 3
    assert Q.shape[0] == K.shape[0] == V.shape[0] # batch_size
    assert Q.shape[2] == K.shape[2] # key_size
    assert K.shape[1] == V.shape[1] # num_values
    scalar_products = Q @ K.transpose(1, 2)
    scalar_products /= K.shape[2]
    weights = F.softmax(scalar_products, dim=1)
    return weights @ V

Запустим эту функцию, передав ей массив из единиц:

In [ ]:
Q = torch.ones((2, 4, 6))
K = torch.ones((2, 8, 6))
V = torch.ones((2, 8, 10))
scaled_dot_product_attention(Q, K, V)

tensor([[[2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2., 2., 2., 2., 2.]],

        [[2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2., 2., 2., 2., 2.]]])

Получился массив из двоек. Но мы считаем взвешенное среднее массива из единиц, как же мы получаем двойки? Так и должно быть, или где-то в коде функции `scaled_dot_product_attention` ошибка? А может быть мы неправильно используем эту функцию?

Если вы считаете, что в коде ошибка, то исправьте ее.

### Решение задания 1

Давайте посмотрим на веса. Для этого в функцию scaled_dot_product_attention добавим печать weights:

In [ ]:
def scaled_dot_product_attention(Q, K, V, masked=False):
    scalar_products = Q @ K.transpose(1, 2)
    scalar_products /= K.shape[2]
    weights = F.softmax(scalar_products, dim=1)
    print(weights.shape, weights)
    return weights @ V

Q = torch.ones((2, 4, 6))
K = torch.ones((2, 8, 6))
V = torch.ones((2, 8, 10))
print(scaled_dot_product_attention(Q, K, V))

Видим, что массив weights имеет размер (batch_size, num_queries, num_values). Здесь все правильно, так и должно быть. Массив weights заполнен значениями 0.25. Получается, что мы считаем взвешенное среднее 8 единичных векторов, используя веса 0.25. Сумма весов получается равной 2, но должна быть равна 1. Значит **в коде функции ошибка**.

Корректно примененная операция softmax всегда должна выдавать значения, сумма которых равна единице. Значит **операция softmax применена некорректно**. Открываем [документацию](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html) по softmax.

> dim (int) – A dimension along which Softmax will be computed (so every slice along dim will sum to 1).

Массив, который мы обрабатываем функцией softmax, имеет размер (2, 4, 8). Последняя ось отвечает за номер Value, поэтому softmax нужно применять по последней оси. В результате мы получим:

`for every i, j: output[i, j, :].sum() == 1`,

что нам и требуется. Значит в softmax надо использовать параметр dim=2 или dim=-1, что то же самое. Исправим код:

In [ ]:
def scaled_dot_product_attention(Q, K, V, masked=False):
    # Исправленный код
    scalar_products = Q @ K.transpose(1, 2)
    scalar_products /= K.shape[2]
    weights = F.softmax(scalar_products, dim=2)
    return weights @ V

Q = torch.ones((2, 4, 6))
K = torch.ones((2, 8, 6))
V = torch.ones((2, 8, 10))
print(scaled_dot_product_attention(Q, K, V))

Теперь функция выдает корректный результат.

### Задание 2

Добавьте возможность маскирования (параметр masked) в функции scaled_dot_product_attention.

Зададим ограничение на входные параметры: при использовании маскирования num_queries должно быть равно num_values.

*Подсказка:* смотрите [теоретический материал](https://limitless-depths-73156.herokuapp.com/Attention_Is_All_You_Need), раздел "Masked attention". Вам нужно реализовать то, что на иллюстрации изображено как *"Masked ("auto-regressive") self-attention"*.

Для проверки используйте следующий код:

In [ ]:
# проверка маскирования в scaled_dot_product_attention

def check(scaled_dot_product_attention):
  _range = torch.arange(4, dtype=torch.float)
  inputs = _range[None, :, None]
  outputs = scaled_dot_product_attention(inputs, inputs, inputs, masked=True)
  assert outputs.shape == (1, 4, 1)
  for i in range(4):
      assert outputs[0, i, 0] == (F.softmax(_range[i]*_range[:i+1], dim=0)*_range[:i+1]).sum()
  print('Check passed!')

### Решение задания 2

`Scalar_products` в функции - это массив размером `(batch_size, num_queries, num_values)`. Для каждого примера в батче это матрица, где строка - номер `query`, столбец - номер `value`.

Чтобы реализовать маскирование, нам нужно заменить некоторые элементы в `scalar_products` на минус бесконечность. Мы заменяем те элементы, где номер столбца больше номера строки, на минус бесконечность. Это можно сделать разными способами. Предпочтительно при этом использовать функции pytorch, а не numpy. Значит для того приходится обращаться к документации pytorch. Например, подойдет такой способ:

In [ ]:
def scaled_dot_product_attention(Q, K, V, masked=False):
    scalar_products = Q @ K.transpose(1, 2)
    scalar_products /= K.shape[2]

    if masked:
      i = torch.arange(scalar_products.shape[1])[:, None]
      j = torch.arange(scalar_products.shape[2])[None, :]
      mask = (i < j)
      scalar_products.masked_fill_(mask, -np.inf)

    weights = F.softmax(scalar_products, dim=2)
    return weights @ V

check(scaled_dot_product_attention)

# Часть 2

В этой части мы будем строить блок энкодера. Изучите следующий код:

### Код блока энкодера

In [ ]:
class EncoderBlock(nn.Module):
    def __init__(self, dim, num_heads=8, dropout=0.1, feedforward_dim=None,
                 kdim=None, vdim=None, autoregressive=False):
        """
        Inputs:
            dim - integer: length of input and output vectors
            num_heads - integer: Number of heads in multihead self-attention,
                                 default: 8
            dropout - float: dropout rate, default 0.1
            feedforward_dim - integer: length of vectors between feedforward
                                       layers, default: 4*dim
            kdim - integer: size of keys, default: dim
            vdim - integer: size of values, default: dim
            autoregressive - boolean: should we use mask? default: False
        """
        super().__init__()

        feedforward_dim = feedforward_dim or 4*dim
        kdim = kdim or dim
        vdim = vdim or dim

        self.self_attention = nn.MultiheadAttention(
            embed_dim=dim,
            num_heads=num_heads,
            batch_first=True,
            kdim=kdim,
            vdim=vdim
        )
        
        self.feedforward = nn.Sequential(
            nn.Linear(dim, feedforward_dim),
            nn.Dropout(dropout),
            nn.ReLU(),
            nn.Linear(feedforward_dim, dim)
        )

        self.norm1 = nn.LayerNorm(dim, eps=1e-5, elementwise_affine=True)
        self.norm2 = nn.LayerNorm(dim, eps=1e-5, elementwise_affine=True)
        self.dropout = nn.Dropout(dropout)
        self.autoregressive = autoregressive

    @staticmethod
    def get_mask(len):
      i = torch.arange(len)[:, None]
      j = torch.arange(len)[None, :]
      return i < j
    
    def forward(self, inputs):
        attn_mask = self.get_mask(inputs.shape[1]) \
                     if self.autoregressive else None
        residual, attn_wights = self.self_attention(inputs, inputs, inputs,
                                                    attn_mask=attn_mask)
        outputs = self.norm1(inputs + self.dropout(residual))

        residual = self.feedforward(outputs)
        outputs = self.norm2(outputs + self.dropout(residual))

        return outputs

Давайте проверим работу блока:

In [ ]:
dim = 512
seq_len = 20

block = EncoderBlock(dim)
inputs = torch.rand((1, seq_len, dim))
outputs = block(inputs)
assert outputs.shape == inputs.shape

Все работает!

### Задание 3

Проверим перестановочную эквивариантность блока. Это ключевое свойство трансформера: перестановка входных векторов приведет к тому, что аналогичным образом будут переставлены выходные вектора, но больше никак эти вектора не изменятся.

In [ ]:
permutation = torch.randperm(seq_len)
print(block(inputs[:, permutation, :]))
print(block(inputs)[:, permutation, :])

Как видим, перестановочная эквивариантность в блоке EncoderBlock не соблюдается: мы получаем разные вектора. Почему? Выясните причину, найдите ошибку и исправьте.

### Решение задания 3

Блок состоит из 2 residual-блоков: self-attention и feedforward. Проведем эксперимент и увидим, что отключение любой из этих частей (комментирование соответствующего участка в методе `forward`) не исправляет ситуацию, то есть **обе части не являются перестановочно эквивариантными**. Неужели у нас двойная проблема? Давайте посмотрим сначала на ту часть, которая проще: на feedforward. Мы сразу видим в ней dropout. Оказалось, что все просто: нам нужно отключить dropout, переведя модель в режим инференса. В итоге мы получим одинаковые результаты:

In [ ]:
block.eval()
permutation = torch.randperm(seq_len)
print(block(inputs[:, permutation, :]))
print(block(inputs)[:, permutation, :])

### Задание 4

В [одной из недавних работ](https://arxiv.org/abs/2002.04745) для более устойчивого обучения предлагается перенести
`LayerNormalization` внутрь residual-блоков. Выполните эту модификацию, поставив `LayerNormalization` после `dropout`.

### Решение задания 4

In [ ]:
def forward(self, inputs):
    residual, attn_wights = self.self_attention(inputs, inputs, inputs)
    outputs = inputs + self.norm1(self.dropout(residual))
    outputs = inputs

    residual = self.feedforward(outputs)
    outputs = outputs + self.norm2(self.dropout(residual))

    return outputs

EncoderBlock.forward = forward

# Часть 3

Эта часть - "Proof of concept". Мы проверим, обучается ли вообще нейросеть, которая построена с помощью нашего блока EncoderBlock. *Спойлер*: она обучается. Сложных заданий по поиску ошибки больше не будет.

### Загрузка данных

In [ ]:
!pip install pytorch-nlp -q
!pip install pytorch_lightning -q
!pip install torch-optimizer -q
!wget -q https://storage.googleapis.com/oleg-zyablov/skillfactory/movie_classification/train.csv

In [ ]:
import pandas as pd
import torch
import math
from sklearn.preprocessing import LabelEncoder
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pytorch_lightning import Trainer
import torch_optimizer as optim
from torchnlp.word_to_vector import GloVe

df = pd.read_csv('train.csv')
texts = df.text.to_numpy() #исходные данные
target = df.genre.to_numpy() #целевые данные

### Задание 5

Проверьте точность случайного угадывания, разделим размер самого часто встречающегося класса на размер всего датасета:

### Решение задания 5

In [ ]:
df.genre.value_counts()[0] / len(df)

Теперь мы знаем, какую точность должна превзойти модель, чтобы мы были уверены, что она хотя бы как-то обучается.

### Продолжаем готовить обучающие данные

- Выполним label-кодирование с помощью `sklearn.preprocessing.LabelEncoder`
- Разобьем тексты на слова (токены)
- Выполним label-кодирование слов с помощью `torchtext.vocab.Vocab`
- Приведем каждый текст к длине в 256 слов добавлением токена `<unk>` или обрезкой
- Создадим `torch.utils.data.TensorDataset`

In [ ]:
target_encoder = LabelEncoder().fit(target)
target = torch.tensor(target_encoder.transform(target), dtype=torch.long)
num_classes = target.max() + 1

tokenizer = get_tokenizer('basic_english')
tokens = [tokenizer(x) for x in texts]

vocab = build_vocab_from_iterator(tokens, specials=['<unk>'], min_freq=10)
vocab.set_default_index(vocab['<unk>'])

token_indices = [torch.tensor(vocab(token_sequence), dtype=torch.long) for token_sequence in tokens]
token_indices = torch.nn.utils.rnn.pad_sequence(token_indices, batch_first=True, padding_value=0)[:, :256]

train_dataset = torch.utils.data.TensorDataset(token_indices[:40000], target[:40000])
val_dataset = torch.utils.data.TensorDataset(token_indices[40000:], target[40000:])

- Загрузим какие-нибудь предобученные эмбеддинги слов, например GloVe (862 MB)
- Создадим веса для слоя Embedding

In [ ]:
vectorizer = GloVe(name='6B', dim=300)
embedding_weights = torch.stack([vectorizer[word] for word in vocab.get_itos()])

### Создаем модели

Будем использовать pytorch-lightning и сравнивать разные модели, начиная от самой простой, и заканчивая трансформером. Сразу переходить к трансформеру было бы неправильно. Вдруг окажется, что простейшая модель выдает точность не хуже трансформера? Иными словами, нам нужен некий бейзлайн.

Вынесем все общие методы, не зависящие от архитектуры модели, в класс GeneralModel.

In [ ]:
# general model for our task
class GeneralModel(pl.LightningModule):
    def training_step(self, batch, batch_idx):
        X, y = batch
        preds = self.forward(X)
        loss = F.cross_entropy(preds, y)
        acc = (preds.argmax(dim=-1) == y).float().mean()
        return {"loss": loss, "acc": acc}
    def training_epoch_end(self, train_step_outputs):
        avg_acc = torch.stack([x['acc'].float() for x in train_step_outputs]).mean()
        print(f'train accuracy {avg_acc.cpu().numpy():g}')
    def validation_step(self, batch, batch_idx):
        X, y = batch
        preds = self.forward(X)
        acc = (preds.argmax(dim=-1) == y).float().mean()
        #self.log('val_acc', acc)
        return {'val_acc': acc}
    def validation_epoch_end(self, validation_step_outputs):
        avg_acc = torch.stack([x['val_acc'].float() for x in validation_step_outputs]).mean()
        print(f'val accuracy {avg_acc.cpu().numpy():g}, ', end='')

Давайте создадим в качестве бейзлайна самую простую модель, какую можно придумать.

In [ ]:
# a simplest model
class SimpleModel(GeneralModel):
    def __init__(self, embedding_weights, num_classes):
        super().__init__()
        embedding_dim = embedding_weights.shape[1]
        self.embedding = nn.Embedding.from_pretrained(embedding_weights, freeze=True)
        self.classifier = nn.Linear(embedding_dim, num_classes)
    def forward(self, inputs):
        raise NotImplementedError #TODO
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

### Задание 6

Напишите метод `forward` в модели `SimpleModel`. Метод должен выполнять 3 шага:
1. Получаем эмбеддинги для входных слов с помощью слоя `self.embedding`. Для этого посмотрите, что модель принимает на вход (то есть на `train_dataset`, созданный ранее), и изучите принцип работы слоя `torch.nn.Embedding`.
2. Усредняем все эмбеддинги. Тем самым получаем вектор фиксированной длины.
3. Применяем классификатор `self.classifier`.

Метод должен возвращать результат работы последнего слоя (классификатора).

Чтобы проверить корректность вашего кода, запустите код ниже из раздела "Обучаем простую модель (бейзлайн)".

### Решение задания 6

In [ ]:
def forward(self, inputs):
    embeddings = self.embedding(inputs)
    mean = torch.mean(embeddings, dim=1)
    logits = self.classifier(mean)
    return logits

SimpleModel.forward = forward

### Обучаем простую модель (бейзлайн)

In [ ]:
simple_model = SimpleModel(embedding_weights, num_classes)

#overfit_batches=0.1, progress_bar_refresh_rate=0, log_every_n_steps=1
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=False)
val_dataloader = DataLoader(val_dataset, batch_size=512, shuffle=False)
Trainer(gpus=1, num_sanity_val_steps=0, max_epochs=5, weights_summary=None).fit(simple_model, train_dataloader, val_dataloader)

Простая модель достигает точности на валидации выше 40%.

### Positional encoding

Для начала нам нужно запрограммировать positional encoding.

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, dim, max_len, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, dim, 2) * (-math.log(10000.0) / dim))
        pe = torch.zeros(max_len, 1, dim)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)
    def forward(self, x):
        x = torch.transpose(x, 0, 1)
        x = x + self.pe[:x.size(0)]
        x = torch.transpose(x, 0, 1)
        return self.dropout(x)

Давайте проверим, что бейзлайн, дополненный positional encoding, не сильно теряет в точности. Конечно использовать positional encoding в модели, которая просто усредняет все эмбеддинги, бессмысленно. Но так мы по крайней мере проверим, что positional encoding ничего не "ломает" и не выдает ошибок. Можно считать это простым юнит-тестом для positional encoding.

In [ ]:
class SimpleModelWithPE(GeneralModel):
    def __init__(self, embedding_weights, num_classes):
        super().__init__()
        embedding_dim = embedding_weights.shape[1]
        self.embedding = nn.Embedding.from_pretrained(embedding_weights, freeze=True)
        self.pos_encoding = PositionalEncoding(embedding_dim, max_len=256, dropout=0.1)
        self.classifier = nn.Linear(embedding_dim, num_classes)
    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        embeddings = self.pos_encoding(embeddings)
        mean = torch.mean(embeddings, dim=1)
        logits = self.classifier(mean)
        return logits
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

simple_model_pe = SimpleModelWithPE(embedding_weights, num_classes)

train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=False)
val_dataloader = DataLoader(val_dataset, batch_size=512, shuffle=False)
Trainer(gpus=1, num_sanity_val_steps=0, max_epochs=5, weights_summary=None).fit(simple_model_pe, train_dataloader, val_dataloader)

### Создаем трансформер

Финал!

Для классификации не нужен декодер - достаточно только энкодера. Обычно для классификации текст дополняют специальным токеном `[CLS]`, и после последнего слоя трансформера считывают результат с этого токена. Затем передают полученный вектор в выходной слой-классификатор. Однако мы пойдем более простым путем, будем просто усреднять все выходные эмбеддинги (как и в простой модели). Усреднение будет выполняться после блоков трансформера.

Для обучения будем использовать оптимизатор [`RAdam`](https://arxiv.org/abs/1908.03265), который отчасти похож на [`Adam`](https://arxiv.org/abs/1412.6980) с warmup.

Ставим `max_epochs=500`. Обучение можно прервать в любой момент.

Конечно трансформер будет обучаться намного дольше, но и достигнет более высокой точности, чем бейзлайн.

In [ ]:
class TransformerPredictor(GeneralModel):
    def __init__(self, embedding_weights, num_classes, num_heads=4, dropout=0.1, lr=1e-3):
        super().__init__()
        model_dim = embedding_weights.shape[1]
        self.embedding = nn.Embedding.from_pretrained(embedding_weights, freeze=True)
        self.pos_encoding = PositionalEncoding(model_dim, max_len=256, dropout=dropout)
        self.cls_embedding = nn.Parameter(torch.zeros(model_dim))
        # Можно использовать либо наш блок EncoderBlock:
        self.encoder = nn.Sequential(*[
            EncoderBlock(dim=model_dim, num_heads=num_heads, dropout=dropout)
            for _ in range(6)
        ])
        # ... либо torch.nn.TransformerEncoder
        # self.encoder = nn.TransformerEncoder(nn.TransformerEncoderLayer(
        #     d_model=model_dim, nhead=num_heads), num_layers=3,
        #     norm=nn.LayerNorm(normalized_shape=model_dim, eps=1e-6))
        self.classifier = nn.Linear(model_dim, num_classes)
    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        embeddings = self.pos_encoding(embeddings)
        encoder_outputs = self.encoder(embeddings)
        encoder_outputs = torch.mean(embeddings, dim=1)
        logits = self.classifier(encoder_outputs)
        return logits
    def configure_optimizers(self):
        return optim.RAdam(self.parameters(), lr=1e-3)

transformer = TransformerPredictor(embedding_weights, num_classes)

train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=False)
val_dataloader = DataLoader(val_dataset, batch_size=512, shuffle=False)
Trainer(gpus=1, num_sanity_val_steps=0, max_epochs=500, log_every_n_steps=1,
        weights_summary=None).fit(transformer, train_dataloader, val_dataloader)